In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [74]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date_unix_timestamp',
                                                   'commit_message','classification', 'fix', 'author_date',
                                                   'contains_bug','fixes',],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    commit_hash = df.commit_hash
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     df.to_csv('data/converted/'+ project + '_understand.csv',index=False)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    df = X
    df['Bugs'] = y
    df['commit_hash'] = commit_hash
    print(df.commit_hash.shape)
    unique_commits = df.commit_hash.unique()
    print(len(unique_commits))
    
#     train_df = df.loc[:df.shape[0]*.6]
#     test_df = df.loc[df.shape[0]*.6:]

    train_df = df[df.commit_hash.isin(unique_commits[0:int(len(unique_commits)*.6)])]
    test_df = df[df.commit_hash.isin(unique_commits[int(len(unique_commits)*.6):])]
    
    train_df.reset_index(inplace=True, drop=True)
    test_df.reset_index(inplace=True, drop=True)
    train_y = train_df.Bugs
    train_X = train_df.drop(['Bugs','commit_hash'],axis = 1)
    
    return train_X,train_y,test_df

def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [79]:
def run_self(project):
    X_train, y_train, test_df = load_data(project)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
#     loc = X_test_w.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = 0
    unique_commits_list = np.array_split(test_df.commit_hash.unique(), 5)
    for i in range(len(unique_commits_list)):
        print("++++++++++++++++++++++++++++++++++")
        test_df_subset = test_df[test_df.commit_hash.isin(unique_commits_list[i])]
        y_test = test_df_subset.Bugs
        X_test = test_df_subset.drop(['Bugs','commit_hash'],axis = 1)
        loc = X_test.CountLineCode                 
        predicted = clf.predict(X_test)
        abcd = metrices.measures(y_test,predicted,loc)
        pf = abcd.get_pf()
        recall = abcd.calculate_recall()
        precision = abcd.calculate_precision()
        f1 = abcd.calculate_f1_score()
        g_score = abcd.get_g_score()
        pci_20 = abcd.get_pci_20()
        ifa = abcd.get_ifa()
        try:
            auc = roc_auc_score(y_test, predicted)
        except:
            auc = 0
        print(classification_report(y_test, predicted))
        print(recall,precision,pf,f1,g_score,auc,pci_20)
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [59]:
def run_self(project):
    X_train, X_test, y_train, y_test = load_data(project)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = 0
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    print(recall,precision,pf,f1,g_score,auc,pci_20)
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [8]:
def run_self_CFS(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    predicted = clf.predict(X_test[cols[:-1]])
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [55]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [80]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'org.alloytools.alloy':
#             continue
        if project != 'guice':
            continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except DivideByZero as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
with open('results/Performance/process+product_nb.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

+++++++++++++++++   guice  +++++++++++++++++
(11613,)
703
++++++++++++++++++++++++++++++++++
              precision    recall  f1-score   support

           0       0.97      0.83      0.90       338
           1       0.80      0.96      0.88       239

    accuracy                           0.89       577
   macro avg       0.89      0.90      0.89       577
weighted avg       0.90      0.89      0.89       577

0.96 0.8 0.17 0.88 0.93 0.8983313114307643 0.61
++++++++++++++++++++++++++++++++++
              precision    recall  f1-score   support

           0       0.95      0.80      0.87       297
           1       0.71      0.92      0.80       153

    accuracy                           0.84       450
   macro avg       0.83      0.86      0.83       450
weighted avg       0.87      0.84      0.85       450

0.92 0.71 0.2 0.8 0.89 0.8598732422261833 0.59
++++++++++++++++++++++++++++++++++
              precision    recall  f1-score   support

           0       0.99      0.87